In [1]:
import pandas as pd

In [4]:
pd.read_csv('data/england-premier-league-matches-2018-to-2019-stats.csv').columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [6]:
pd.read_csv('data/england-premier-league-teams-2018-to-2019-stats.csv').columns.to_list()

['team_name',
 'common_name',
 'season',
 'country',
 'matches_played',
 'matches_played_home',
 'matches_played_away',
 'suspended_matches',
 'wins',
 'wins_home',
 'wins_away',
 'draws',
 'draws_home',
 'draws_away',
 'losses',
 'losses_home',
 'losses_away',
 'points_per_game',
 'points_per_game_home',
 'points_per_game_away',
 'league_position',
 'league_position_home',
 'league_position_away',
 'performance_rank',
 'goals_scored',
 'goals_conceded',
 'goal_difference',
 'total_goal_count',
 'total_goal_count_home',
 'total_goal_count_away',
 'goals_scored_home',
 'goals_scored_away',
 'goals_conceded_home',
 'goals_conceded_away',
 'goal_difference_home',
 'goal_difference_away',
 'minutes_per_goal_scored',
 'minutes_per_goal_scored_home',
 'minutes_per_goal_scored_away',
 'minutes_per_goal_conceded',
 'minutes_per_goal_conceded_home',
 'minutes_per_goal_conceded_away',
 'clean_sheets',
 'clean_sheets_home',
 'clean_sheets_away',
 'btts_count',
 'btts_count_home',
 'btts_count_awa